In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import numpy as np
from PIL import Image
import io
import cv2
import time

In [6]:
def lidar_to_2d_front_view(points,
                           v_res,
                           h_res,
                           v_fov,
                           val="depth",
                           cmap="jet",
                           saveto=None,
                           y_fudge=0.0
                           ):
    """ Takes points in 3D space from LIDAR data and projects them to a 2D
        "front view" image, and saves that image.

    Args:
        points: (np array)
            The numpy array containing the lidar points.
            The shape should be Nx4
            - Where N is the number of points, and
            - each point is specified by 4 values (x, y, z, reflectance)
        v_res: (float)
            vertical resolution of the lidar sensor used.
        h_res: (float)
            horizontal resolution of the lidar sensor used.
        v_fov: (tuple of two floats)
            (minimum_negative_angle, max_positive_angle)
        val: (str)
            What value to use to encode the points that get plotted.
            One of {"depth", "height", "reflectance"}
        cmap: (str)
            Color map to use to color code the `val` values.
            NOTE: Must be a value accepted by matplotlib's scatter function
            Examples: "jet", "gray"
        saveto: (str or None)
            If a string is provided, it saves the image as this filename.
            If None, then it just shows the image.
        y_fudge: (float)
            A hacky fudge factor to use if the theoretical calculations of
            vertical range do not match the actual data.

            For a Velodyne HDL 64E, set this value to 5.
    """

    # DUMMY PROOFING
    assert len(v_fov) ==2, "v_fov must be list/tuple of length 2"
    assert v_fov[0] <= 0, "first element in v_fov must be 0 or negative"
    #assert val in {"depth", "height", "intensity"}, \
        #'val must be one of {"depth", "height", "intensity"}'


    x_lidar = points[:, 0]
    y_lidar = points[:, 1]
    z_lidar = points[:, 2]
    r_lidar = points[:, 3] # Reflectance
    # Distance relative to origin when looked from top
    d_lidar = np.sqrt(x_lidar ** 2 + y_lidar ** 2)
    # Absolute distance relative to origin
    d_lidar_abs = np.sqrt(x_lidar ** 2 + y_lidar ** 2, z_lidar ** 2)

    v_fov_total = -v_fov[0] + v_fov[1]

    # Convert to Radians
    v_res_rad = v_res * (np.pi/180)
    h_res_rad = h_res * (np.pi/180)

    # PROJECT INTO IMAGE COORDINATES
    x_img = np.arctan2(-y_lidar, x_lidar)/ h_res_rad #theta 
    y_img = np.arctan2(z_lidar, d_lidar_abs)/ v_res_rad #phi


    # SHIFT COORDINATES TO MAKE 0,0 THE MINIMUM
    x_min = -360.0 / h_res / 2  # Theoretical min x value based on sensor specs
    x_img -= x_min              # Shift
    x_max = 360.0 / h_res       # Theoretical max x value after shifting

    y_min = v_fov[0] / v_res    # theoretical min y value based on sensor specs
    y_img -= y_min              # Shift
    y_max = v_fov_total / v_res # Theoretical max x value after shifting

    y_max += y_fudge            # Fudge factor if the calculations based on
                                # spec sheet do not match the range of
                                # angles collected by in the data.

    # WHAT DATA TO USE TO ENCODE THE VALUE FOR EACH PIXEL
    if val == "intensity":
        pixel_values = r_lidar
    elif val == "height":
        pixel_values = z_lidar
    elif val == "x":
        pixel_values = x_lidar
    elif val == "y":
        pixel_values = y_lidar
    elif val == "z":
        pixel_values = z_lidar
    
    else:
        pixel_values = d_lidar

    pixel_values=np.array(pixel_values)

    # Rescale x_img and y_img to fit within 64x1024 grid
    x_img_scaled = (x_img / x_max) * 1024
    y_img_scaled = (y_img / y_max) * 64


    # Create a 2D array to store pixel values
    image_array = np.zeros((64, 1024))

    # Assign pixel values to the corresponding grid points
    for i in range(len(x_img_scaled)):
        x_index = int(round(x_img_scaled[i]))
        y_index = int(round(y_img_scaled[i]))
        if 0 <= x_index < 1024 and 0 <= y_index < 64:
            image_array[63-y_index, x_index] = pixel_values[i]

    if saveto is not None:
        np.save(saveto, image_array)


In [7]:
# set input and output folder paths
folder = 'Train'
input_folder = rf"/DATA2/Vivek/Data/Csv/csv files/Subset_Downstream_Task/{folder}"
output_folder = rf"/DATA2/Vivek/Data/Downstream Task/{folder}/train_lidar_x"

# create output folder if it does not exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [8]:
HRES = 0.35         # horizontal resolution (assuming 20Hz setting)
VRES = 0.4          # vertical res
#VFOV = (-25, 15) # Field of view (-ve, +ve) along vertical axis
VFOV = (-25, 2) # Field of view (-ve, +ve) along vertical axis
Y_FUDGE = 0         # y fudge factor for velodyne HDL 64E

#val must be one of {"depth", "height", "intensity","angle"}
#value = "intensity"
value = "x"


all_files = [filename for filename in os.listdir(input_folder) if filename.endswith('.csv')]
total_files = len(all_files)
start_time = time.time()

for index, filename in enumerate(all_files):
    # read CSV file into pandas dataframe
    df = pd.read_csv(os.path.join(input_folder, filename))
    
    # convert dataframe to numpy array
    lidar = np.array(df[['x', 'y', 'z', 'intensity']])
    
    # apply lidar_to_2d_front_view function
    output_filename = os.path.splitext(filename)[0] + '.npy'
    output_path = os.path.join(output_folder, output_filename)
    lidar_to_2d_front_view(lidar, v_res=VRES, h_res=HRES, v_fov=VFOV, val=value, saveto=output_path, y_fudge=Y_FUDGE)
    
    processed_files = index + 1
    elapsed_time = time.time() - start_time
    estimated_time_remaining = elapsed_time / processed_files * (total_files - processed_files)
    print(f"Processed {processed_files} out of {total_files}. Estimated time remaining: {estimated_time_remaining:.2f} seconds.")

Processed 1 out of 5000. Estimated time remaining: 3050.74 seconds.
Processed 2 out of 5000. Estimated time remaining: 3022.45 seconds.
Processed 3 out of 5000. Estimated time remaining: 3064.50 seconds.
Processed 4 out of 5000. Estimated time remaining: 3116.32 seconds.
Processed 5 out of 5000. Estimated time remaining: 3127.90 seconds.
Processed 6 out of 5000. Estimated time remaining: 3132.52 seconds.
Processed 7 out of 5000. Estimated time remaining: 3116.20 seconds.
Processed 8 out of 5000. Estimated time remaining: 3105.25 seconds.
Processed 9 out of 5000. Estimated time remaining: 3109.77 seconds.
Processed 10 out of 5000. Estimated time remaining: 2829.45 seconds.
Processed 11 out of 5000. Estimated time remaining: 2785.35 seconds.
Processed 12 out of 5000. Estimated time remaining: 2669.92 seconds.
Processed 13 out of 5000. Estimated time remaining: 2573.35 seconds.
Processed 14 out of 5000. Estimated time remaining: 2424.81 seconds.
Processed 15 out of 5000. Estimated time re